<a href="https://colab.research.google.com/github/datajcthemax/playdata/blob/main/day22_K_Clustering%EA%B3%BC_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 22번째 이야기
- K-Clustering과 PCA
- about 준지도학습
- TabNet
- 머신러닝 시작부터 끝까지 질의응답 미팅

In [ ]:
pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 46.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 w

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.__version__

'1.13.1+cu117'

In [ ]:
!nvidia-smi

Tue Apr 25 01:23:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np
PATH = '/content/drive/MyDrive/datas/따릉이/train.csv'
data = pd.read_csv(PATH)

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1328 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1328 non-null   int64  
 1   hour                    1328 non-null   int64  
 2   hour_bef_temperature    1328 non-null   float64
 3   hour_bef_precipitation  1328 non-null   float64
 4   hour_bef_windspeed      1328 non-null   float64
 5   hour_bef_humidity       1328 non-null   float64
 6   hour_bef_visibility     1328 non-null   float64
 7   hour_bef_ozone          1328 non-null   float64
 8   hour_bef_pm10           1328 non-null   float64
 9   hour_bef_pm2.5          1328 non-null   float64
 10  count                   1328 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 124.5 KB


In [ ]:
y = data.pop('count')
X = data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_train, X_test, y_train, y_test = X_train, X_test, y_train.to_numpy(), y_test.to_numpy()

In [ ]:
X_train.dtype

dtype('float64')

In [ ]:
from pytorch_tabnet.pretraining import TabNetPretrainer
# TabNetPretrainer
unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    mask_type='entmax', # "sparsemax"
)

unsupervised_model.fit(
    X_train=X_train,
    eval_set=[X_test],
    pretraining_ratio=0.8,
    batch_size=16
)


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 80.40638| val_0_unsup_loss_numpy: 8.55443000793457|  0:00:02s
epoch 1  | loss: 54.03213| val_0_unsup_loss_numpy: 9.613380432128906|  0:00:05s
epoch 2  | loss: 36.08475| val_0_unsup_loss_numpy: 8.623820304870605|  0:00:07s
epoch 3  | loss: 24.59331| val_0_unsup_loss_numpy: 8.043729782104492|  0:00:09s
epoch 4  | loss: 15.23244| val_0_unsup_loss_numpy: 9.765270233154297|  0:00:11s
epoch 5  | loss: 11.61294| val_0_unsup_loss_numpy: 8.98369026184082|  0:00:13s
epoch 6  | loss: 8.81305 | val_0_unsup_loss_numpy: 5.680079936981201|  0:00:15s
epoch 7  | loss: 6.34877 | val_0_unsup_loss_numpy: 5.152190208435059|  0:00:18s
epoch 8  | loss: 4.32741 | val_0_unsup_loss_numpy: 4.222350120544434|  0:00:21s
epoch 9  | loss: 3.03826 | val_0_unsup_loss_numpy: 3.0964999198913574|  0:00:23s
epoch 10 | loss: 2.27803 | val_0_unsup_loss_numpy: 3.141239881515503|  0:00:25s
epoch 11 | loss: 2.18571 | val_0_unsup_loss_numpy: 2.592360019683838|  0:00:27s
epoch 12 | loss: 1.56009 | val_0_unsup_lo

/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor

clf = TabNetRegressor(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    scheduler_params={"step_size":10, # how to use learning rate scheduler
                      "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax' # This will be overwritten if using pretrain model
)

clf.fit(
    X_train=X_train, y_train=y_train.reshape(-1,1),
    eval_set=[(X_train, y_train.reshape(-1,1)), (X_test, y_test.reshape(-1,1))],
    eval_name=['train', 'valid'],
    eval_metric=['rmse'],
    from_unsupervised=unsupervised_model,batch_size=8
)

/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/abstract_model.py:105: UserWarning: Pretraining: mask_type changed from sparsemax to entmax
  warnings.warn(wrn_msg)
/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/abstract_model.py:231: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")


epoch 0  | loss: 18801.74652| train_rmse: 137.17806| valid_rmse: 132.45109|  0:00:04s
epoch 1  | loss: 18260.55694| train_rmse: 134.48133| valid_rmse: 130.22859|  0:00:08s
epoch 2  | loss: 17220.00829| train_rmse: 130.88515| valid_rmse: 126.96969|  0:00:14s
epoch 3  | loss: 15856.55357| train_rmse: 124.90751| valid_rmse: 121.20262|  0:00:18s
epoch 4  | loss: 14258.08502| train_rmse: 116.97406| valid_rmse: 113.73529|  0:00:22s
epoch 5  | loss: 12381.29668| train_rmse: 108.27731| valid_rmse: 104.89527|  0:00:27s
epoch 6  | loss: 10620.5443| train_rmse: 99.16713| valid_rmse: 96.04575|  0:00:32s
epoch 7  | loss: 8848.18018| train_rmse: 90.63128| valid_rmse: 87.43726|  0:00:36s
epoch 8  | loss: 7250.82282| train_rmse: 82.03851| valid_rmse: 79.47069|  0:00:41s
epoch 9  | loss: 6236.39324| train_rmse: 75.02666| valid_rmse: 72.84199|  0:00:46s
epoch 10 | loss: 5406.44365| train_rmse: 69.69316| valid_rmse: 67.36945|  0:00:50s
epoch 11 | loss: 4987.39691| train_rmse: 65.49371| valid_rmse: 63.282

In [ ]:
from sklearn.metrics import r2_score
pred = clf.predict(X_test)

In [ ]:
r2_score(y_test, pred)